# SPTAG python wrapper tutorial 

To end-to-end build a vector search online service, it contains two steps:
- Offline build SPTAG index for database vectors
- Online serve the index to support vector search requests from the clients

## Offline build SPTAG index

> Prepare input vectors and metadatas for SPTAG

In [2]:
import os
import numpy as np

vector_number = 100
vector_dimension = 100

# Randomly generate the database vectors. Currently SPTAG only support int8, int16 and float32 data type.
x = np.random.rand(vector_number, vector_dimension).astype(np.float32) 

# Prepare metadata for each vectors, separate them by '\n'. Currently SPTAG python wrapper only support '\n' as the separator
m = ''
for i in range(vector_number):
    m += str(i) + '\n'

> Build SPTAG index for database vectors **x**

In [3]:
import SPTAG

index = SPTAG.AnnIndex('BKT', 'Float', vector_dimension)

# Set the thread number to speed up the build procedure in parallel 
index.SetBuildParam("NumberOfThreads", '4', "Index")

# Set the distance type. Currently SPTAG only support Cosine and L2 distances. Here Cosine distance is not the Cosine similarity. The smaller Cosine distance it is, the better.
index.SetBuildParam("DistCalcMethod", 'Cosine', "Index") 

if index.BuildWithMetaData(x, m, vector_number, False, False):
    index.Save("sptag_index") # Save the index to the disk

os.listdir('sptag_index')

['deletes.bin',
 'graph.bin',
 'indexloader.ini',
 'metadata.bin',
 'metadataIndex.bin',
 'tree.bin',
 'vectors.bin']

In [4]:
# Local index test on the vector search
index = SPTAG.AnnIndex.Load('sptag_index')

# prepare query vector
q = np.random.rand(vector_dimension).astype(np.float32)

result = index.SearchWithMetaData(q, 3) # Search k=3 nearest vectors for query vector q
print (result[0]) # nearest k vector ids
print (result[1]) # nearest k vector distances
print (result[2]) # nearest k vector metadatas

[84, 3, 37]
[-3.7501039505004883, -3.6583847999572754, -3.6582584381103516]
[b'84\n', b'3\n', b'37\n']


> Build SPTAG index for database vector **x** with PQ quantization

In [5]:
import subprocess
import struct
import numpy as np

cmd = "Quantizer.exe -d %d -v Float -f DEFAULT -i sptag_index\\vectors.bin -o quan_doc_vectors.bin -oq quantizer.bin -qt PQQuantizer -qd %d -ts %d -norm true" % (vector_dimension, int(vector_dimension / 2), vector_number)
result = subprocess.check_output(cmd, stderr=subprocess.STDOUT)
print (result) 
# For SPTAG index with quantization:
#     Cosine distance: norm -> true and use L2 for index build
#     L2 distance: norm -> false and use L2 for index build

f = open('quan_doc_vectors.bin', 'rb')
r = struct.unpack('i', f.read(4))[0]
c = struct.unpack('i', f.read(4))[0]
quan_x = np.frombuffer(f.read(r*c), dtype=np.uint8).reshape((r,c))
print (quan_x.shape)

b'[1] Using AVX2 InstructionSet!\r\n[1] Normalizing vectors.\r\n[1] Quantizer Does not exist. Training a new one.\r\n[1] Begin Training Quantizer Codebooks.\r\n[1] [1] Training Codebook 22.\r\nTraining Codebook 26.\r\n[1] Training Codebook 5.\r\n[1] Training Codebook 0.\r\n[1] Training Codebook 14.\r\n[1] Training Codebook 30.\r\n[1] Training Codebook 18.\r\n[1] Training Codebook 42.\r\n[1] Training Codebook 10.\r\n[1] Training Codebook 46.\r\n[1] Training Codebook 34.\r\n[1] Training Codebook 38.\r\n[1] Lambda:min(inf,2.68353e-08) Max:1 Min:0 Avg:0.390625 Std/Avg:1.249000 Dist:0.000000 NonZero/Total:100/256\r\n1\t1\t1\t1\t1\t1\t1\t1\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t0\t1\t1\t1\t0\t1\t0\t0\t0\t0\t1\t0\t1\t0\t1\t0\t0\t1\t0\t0\t1\t1\t0\t0\t0\t0\t0\t0\t1\t0\t0\t0\t0\t0\t1\t0\t1\t0\t1\t0\t0\t1\t0\t1\t0\t0\t1\t0\t1\t0\t1\t0\t1\t1\t0\t0\t0\t0\t0\t0\t0\t0\t0\t1\t1\t0\t0\t0\t1\t0\t0\t0\t1\t1\t1\t0\t1\t0\t0\t0\t0\t1\t1\t0\t0\t0\t0\t0\t1\t1\t1\t0\t1\t1\t0\t0\t1\t1\t1\t1\t1\t0\t0\t0\t1\t0\t0\t

In [13]:
import SPTAG

index = SPTAG.AnnIndex('BKT', 'Float', int(vector_dimension / 2))

# Set the thread number to speed up the build procedure in parallel 
index.SetBuildParam("NumberOfThreads", '4', "Index")

# Set the distance type. Currently SPTAG only support Cosine and L2 distances. Here Cosine distance is not the Cosine similarity. The smaller Cosine distance it is, the better.
index.SetBuildParam("DistCalcMethod", 'Cosine', "Index") 

if index.LoadQuantizer("quantizer.bin") and index.BuildWithMetaData(quan_x, m, vector_number, False, False):
    index.Save("quan_sptag_index") # Save the index to the disk

os.listdir('quan_sptag_index')

['deletes.bin',
 'graph.bin',
 'indexloader.ini',
 'metadata.bin',
 'metadataIndex.bin',
 'quantizer.bin',
 'tree.bin',
 'vectors.bin']

In [14]:
# Local index test on the vector search
index = SPTAG.AnnIndex.Load('quan_sptag_index')
index.SetQuantizerADC(True)
result = index.SearchWithMetaData(q, 12) # Search k=3 nearest vectors for query vector q
print (result[0]) # nearest k vector ids
print (result[1]) # nearest k vector distances
print (result[2]) # nearest k vector metadatas

[676, 260, 701, 263, 285, 479, 520, 753, 764, 795, 890, 185, 607, 9, 137, 295, 398, 561, 569, 801, 846, 291, 89, 159]
[0.10097091645002365, 4.623202303216714e+19, 4.623202303216714e+19, 2.58399095169963e+20, 2.58399095169963e+20, 2.58399095169963e+20, 2.58399095169963e+20, 2.58399095169963e+20, 2.58399095169963e+20, 2.58399095169963e+20, 2.58399095169963e+20, 1.7926470230357202e+22, 1.7926470230357202e+22, 6.847471932557632e+22, 6.847471932557632e+22, 6.847471932557632e+22, 6.847471932557632e+22, 6.847471932557632e+22, 6.847471932557632e+22, 6.847471932557632e+22, 6.847471932557632e+22, 6.858322905499818e+22, 6.8608197011332326e+22, 6.8608197011332326e+22]
[b'676\n', b'260\n', b'701\n', b'263\n', b'285\n', b'479\n', b'520\n', b'753\n', b'764\n', b'795\n', b'890\n', b'185\n', b'607\n', b'9\n', b'137\n', b'295\n', b'398\n', b'561\n', b'569\n', b'801\n', b'846\n', b'291\n', b'89\n', b'159\n']


## Online serve the index

Start the vector search service on the host machine which listens for the client requests on the port 8000

> Write a server configuration file **service.ini** as follows:

```bash
[Service]
ListenAddr=0.0.0.0
ListenPort=8000
ThreadNumber=8
SocketThreadNumber=8

[QueryConfig]
DefaultMaxResultNumber=6
DefaultSeparator=|

[Index]
List=MyIndex

[Index_MyIndex]
IndexFolder=sptag_index
```

> Start the server on the host machine

```bash
Server.exe -m socket -c service.ini
```

It will print the follow messages:

```bash
Setting TreeFilePath with value tree.bin
Setting GraphFilePath with value graph.bin
Setting VectorFilePath with value vectors.bin
Setting DeleteVectorFilePath with value deletes.bin
Setting BKTNumber with value 1
Setting BKTKmeansK with value 32
Setting BKTLeafSize with value 8
Setting Samples with value 1000
Setting TPTNumber with value 32
Setting TPTLeafSize with value 2000
Setting NumTopDimensionTpTreeSplit with value 5
Setting NeighborhoodSize with value 32
Setting GraphNeighborhoodScale with value 2
Setting GraphCEFScale with value 2
Setting RefineIterations with value 2
Setting CEF with value 1000
Setting MaxCheckForRefineGraph with value 8192
Setting NumberOfThreads with value 4
Setting DistCalcMethod with value Cosine
Setting DeletePercentageForRefine with value 0.400000
Setting AddCountForRebuild with value 1000
Setting MaxCheck with value 8192
Setting ThresholdOfNumberOfContinuousNoBetterPropagation with value 3
Setting NumberOfInitialDynamicPivots with value 50
Setting NumberOfOtherDynamicPivots with value 4
Load Vector From sptag_index\vectors.bin
Load Vector (100, 10) Finish!
Load BKT From sptag_index\tree.bin
Load BKT (1,101) Finish!
Load Graph From sptag_index\graph.bin
Load Graph (100, 32) Finish!
Load DeleteID From sptag_index\deletes.bin
Load DeleteID (100, 1) Finish!
Start to listen 0.0.0.0:8000 ...
```

> Start python client to connect to the server and send vector search request.

In [4]:
import SPTAGClient
import time

# connect to the server
client = SPTAGClient.AnnClient('127.0.0.1', '8000')
while not client.IsConnected():
    time.sleep(1)
client.SetTimeoutMilliseconds(18000)

k = 3
vector_dimension = 10
# prepare query vector
q = np.random.rand(vector_dimension).astype(np.float32)

result = client.Search(q, k, 'Float', True) # AnnClient.Search(query_vector, knn, data_type, with_metadata)

print (result[0]) # nearest k vector ids
print (result[1]) # nearest k vector distances
print (result[2]) # nearest k vector metadatas



[15, 35, 75]
[-1.094923973083496, -1.0913333892822266, -1.0795197486877441]
['15\n', '35\n', '75\n']
